# End-to-end OptiMUS pipeline

This notebook runs the full flow: **problem description + data** → **OptiMUS inputs** → **OptiMUS run** → **solution**.

1. **Setup** – paths and imports  
2. **Inputs** – paths to data and description  
3. **Step 1** – Generate OptiMUS inputs (desc.txt, params.json, labels.json) from description + data  
4. **Step 2** – Run OptiMUS pipeline (extract objective, constraints, generate code, solve)  
5. **Step 3** – Load and display the solution

In [17]:
# Setup: project root and imports
import os
import sys
import subprocess
from pathlib import Path

# Project root (parent of scripts/)
NOTEBOOK_DIR = Path.cwd()
PROJECT_ROOT = NOTEBOOK_DIR.parent if (NOTEBOOK_DIR / "data_to_optimus.py").exists() else NOTEBOOK_DIR
os.chdir(PROJECT_ROOT)
sys.path.insert(0, str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")

Project root: /Users/hindy/Desktop/OptiMUS


## Inputs

Set paths to your **data file(s)** (CSV/Excel) and **problem description** (text or path to `.txt`). Use a single path or a list of paths for multiple datasets (e.g. `DATA_PATH = ["inventory.csv", "stores.csv"]`). Output directory is where OptiMUS input files will be written.

In [19]:
# Paths (relative to project root)
# Data and description can be anywhere; output goes to current_query so OptiMUS runs with no --dir
DATA_PATH = "/Users/hindy/Desktop/OptiMUS/operations_data/inventory_monitoring.csv"
DESCRIPTION_PATH = "/Users/hindy/Desktop/OptiMUS/operations_data/desc.txt"
OUTPUT_DIR = "current_query"
MODEL = "gpt-4o-mini"

# Optional: use --no-llm for no API calls, or --simple for one param per column
USE_EXPERT = True

## Step 1: Generate OptiMUS inputs

Convert **description + data** into `desc.txt`, `params.json`, and `labels.json` in the output directory. Uses expert (LLM) extraction by default to map columns to parameters.

In [20]:
import importlib.util
_spec = importlib.util.spec_from_file_location(
    "data_to_optimus",
    Path(PROJECT_ROOT) / "scripts" / "data_to_optimus.py",
)
_data_to_optimus = importlib.util.module_from_spec(_spec)
_spec.loader.exec_module(_data_to_optimus)

_data_to_optimus.run(
    DATA_PATH,
    DESCRIPTION_PATH,
    output_dir=OUTPUT_DIR,
    model=MODEL,
    use_expert=USE_EXPERT,
    no_llm=False,
)
print("OptiMUS inputs written to", OUTPUT_DIR)

Loaded 31 rows, 9 columns from /Users/hindy/Desktop/OptiMUS/operations_data/inventory_monitoring.csv
Extracting parameters with expert (LLM) reasoning over description and dataset...
Wrote OptiMUS inputs to current_query/
  desc.txt, params.json, labels.json (9 parameters)

Next: run OptiMUS with:
  python optimus.py
OptiMUS inputs written to current_query


## Step 2: Run OptiMUS pipeline

Run the full OptiMUS pipeline on the generated folder: extract objective and constraints, formulate them, generate Gurobi code, execute and debug until a solution is found.

In [21]:
from optimus import run_pipeline

state = run_pipeline(
    problem_dir=OUTPUT_DIR,
    model=MODEL,
    error_correction=True,
)
print("Pipeline finished. State keys:", list(state.keys()))

{'description': '"The goal is to reduce stockouts on high-demand products, minimize holding and markdown costs, and avoid costly emergency shipments."', 'formulation': None, 'code': None}
_________________________ get_constraints _________________________
[{'description': 'Stock levels for each product at each store must be non-negative', 'formulation': None, 'code': None}, {'description': 'Total inventory at each warehouse must not exceed the warehouse capacity', 'formulation': None, 'code': None}, {'description': 'Reorder points for each product must be set above the minimum stock level to avoid stockouts', 'formulation': None, 'code': None}, {'description': 'Product allocations from warehouses to stores must not exceed the current stock levels', 'formulation': None, 'code': None}, {'description': 'Supplier lead times must be accounted for in the replenishment schedule', 'formulation': None, 'code': None}, {'description': 'Expiry dates must be considered to manage perishable items ef

KeyboardInterrupt: 

## Step 3: Load and display the solution

Read the solver output and optional solution file from `output/`.

In [13]:
output_dir = Path(PROJECT_ROOT) / OUTPUT_DIR / "output"

# Solver stdout (often contains "Optimal Objective Value: ...")
code_output = output_dir / "code_output.txt"
if code_output.exists():
    print("--- code_output.txt ---")
    print(code_output.read_text())
else:
    print("code_output.txt not found")

# Written optimal value (if status was OPTIMAL)
solution_file = output_dir / "output_solution.txt"
if solution_file.exists():
    print("\n--- output_solution.txt ---")
    print(solution_file.read_text())

code_output.txt not found
